In [1]:
from IPython.display import HTML

## Testing recognition from bash - DEMO

In [2]:
!python mmaction2/demo/demo_skeleton.py data/uji_jalan.mp4 data/uji_jalan_out_demo.mp4 \
    --config mmaction2/configs/skeleton/posec3d/slowonly_r50_8xb16-u48-240e_ntu60-xsub-keypoint.py \
    --checkpoint https://download.openmmlab.com/mmaction/skeleton/posec3d/slowonly_r50_u48_240e_ntu60_xsub_keypoint/slowonly_r50_u48_240e_ntu60_xsub_keypoint-f3adabf1.pth \
    --det-config mmaction2/demo/demo_configs/faster-rcnn_r50_fpn_2x_coco_infer.py \
    --det-checkpoint http://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_2x_coco/faster_rcnn_r50_fpn_2x_coco_bbox_mAP-0.384_20200504_210434-a5d8aa15.pth \
    --det-score-thr 0.9 \
    --det-cat-id 0 \
    --pose-config mmaction2/demo/demo_configs/td-hm_hrnet-w32_8xb64-210e_coco-256x192_infer.py \
    --pose-checkpoint https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w32_coco_256x192-c78dce93_20200708.pth \
    --label-map mmaction2/tools/data/skeleton/label_map_ntu60.txt

Loads checkpoint by http backend from path: http://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_2x_coco/faster_rcnn_r50_fpn_2x_coco_bbox_mAP-0.384_20200504_210434-a5d8aa15.pth
Performing Human Detection for each frame
[                                                  ] 0/319, elapsed: 0s, ETA:02/19 00:11:24 - mmengine - WARNING - "FileClient" will be deprecated in future. Please use io functions in https://mmengine.readthedocs.io/en/latest/api/fileio.html#file-io
02/19 00:11:24 - mmengine - WARNING - "HardDiskBackend" is the alias of "LocalBackend" and the former will be deprecated in future.
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 319/319, 4.1 task/s, elapsed: 78s, ETA:     0s
Loads checkpoint by http backend from path: https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w32_coco_256x192-c78dce93_20200708.pth
Performing Human Pose Estimation for each frame
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 319/319, 15.8 task/s, elapsed: 20s, ETA:     0s
Loads checkpoint by ht

In [3]:
HTML('<video width=50% controls autoplay loop><source src="data/uji_jalan.mp4"></video>')

In [4]:
HTML('<video width=50% controls autoplay loop><source src="data/uji_jalan_out_demo.mp4"></video>')

## Testing recognition from python - DEMO

In [1]:
import tempfile

import cv2
import mmcv
import mmengine
import torch
from mmengine.utils import track_iter_progress

from mmaction.apis import (detection_inference, inference_skeleton,
                           init_recognizer, pose_inference)
from mmaction.registry import VISUALIZERS
from mmaction.utils import frame_extract

import moviepy.editor as mpy

In [2]:
import moviepy.config as cf
print ( cf.get_setting("FFMPEG_BINARY") ) # prints the current setting, make sure to use imageio_ffmpeg

/home/aldy/miniconda3/envs/openmmlab/lib/python3.8/site-packages/imageio_ffmpeg/binaries/ffmpeg-linux64-v4.2.2


In [7]:
FONTFACE = cv2.FONT_HERSHEY_DUPLEX
FONTSCALE = 0.75
FONTCOLOR = (255, 255, 255)  # BGR, white
THICKNESS = 1
LINETYPE = 1

def visualize(pose_config, out_filename, frames, data_samples, action_label):
    pose_config = mmengine.Config.fromfile(pose_config)
    visualizer = VISUALIZERS.build(pose_config.visualizer)
    visualizer.set_dataset_meta(data_samples[0].dataset_meta)

    vis_frames = []
    print('Drawing skeleton for each frame')
    for d, f in track_iter_progress(list(zip(data_samples, frames))):
        f = mmcv.imconvert(f, 'bgr', 'rgb')
        visualizer.add_datasample(
            'result',
            f,
            data_sample=d,
            draw_gt=False,
            draw_heatmap=False,
            draw_bbox=True,
            show=False,
            wait_time=0,
            out_file=None,
            kpt_thr=0.3)
        vis_frame = visualizer.get_image()
        cv2.putText(vis_frame, action_label, (10, 30), FONTFACE, FONTSCALE,
                    FONTCOLOR, THICKNESS, LINETYPE)
        vis_frames.append(vis_frame)

    vid = mpy.ImageSequenceClip(vis_frames, fps=24)
    vid.write_videofile(out_filename, remove_temp=True)

In [8]:
video = 'data/uji_jalan.mp4'
out_filename = 'data/uji_jalan_out_demo.mp4'

# Choose to use an action classification config
config = 'mmaction2/configs/skeleton/posec3d/slowonly_r50_8xb16-u48-240e_ntu60-xsub-keypoint.py'
# Setup a checkpoint file to load
checkpoint = 'https://download.openmmlab.com/mmaction/skeleton/posec3d/slowonly_r50_u48_240e_ntu60_xsub_keypoint/slowonly_r50_u48_240e_ntu60_xsub_keypoint-f3adabf1.pth'

# human detection config
det_config = 'mmaction2/demo/demo_configs/faster-rcnn_r50_fpn_2x_coco_infer.py'
det_checkpoint = 'http://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_2x_coco/faster_rcnn_r50_fpn_2x_coco_bbox_mAP-0.384_20200504_210434-a5d8aa15.pth'
det_score_thr = 0.9
det_cat_id = 0

# pose estimation config
pose_config = 'mmaction2/demo/demo_configs/td-hm_hrnet-w32_8xb64-210e_coco-256x192_infer.py'
pose_checkpoint = 'https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w32_coco_256x192-c78dce93_20200708.pth'
label_map = 'mmaction2/tools/data/skeleton/label_map_ntu60.txt'

In [9]:
device = 'cuda:0'
short_side = 480

In [10]:
tmp_dir = tempfile.TemporaryDirectory()
frame_paths, frames = frame_extract(video, short_side,
                                    tmp_dir.name)

h, w, _ = frames[0].shape

In [11]:
# Get Human detection results.
det_results, _ = detection_inference(det_config, det_checkpoint,
                                     frame_paths, det_score_thr,
                                     det_cat_id, device)
torch.cuda.empty_cache()

Loads checkpoint by http backend from path: http://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_2x_coco/faster_rcnn_r50_fpn_2x_coco_bbox_mAP-0.384_20200504_210434-a5d8aa15.pth
Performing Human Detection for each frame
[                                                  ] 0/319, elapsed: 0s, ETA:02/19 00:20:20 - mmengine - WARNING - "FileClient" will be deprecated in future. Please use io functions in https://mmengine.readthedocs.io/en/latest/api/fileio.html#file-io
02/19 00:20:20 - mmengine - WARNING - "HardDiskBackend" is the alias of "LocalBackend" and the former will be deprecated in future.
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 319/319, 4.0 task/s, elapsed: 80s, ETA:     0s


In [12]:
# Get Pose estimation results.
pose_results, pose_data_samples = pose_inference(pose_config,
                                                 pose_checkpoint,
                                                 frame_paths, det_results,
                                                 device)
torch.cuda.empty_cache()

Loads checkpoint by http backend from path: https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w32_coco_256x192-c78dce93_20200708.pth
Performing Human Pose Estimation for each frame
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 319/319, 13.3 task/s, elapsed: 24s, ETA:     0s


In [13]:
# Initialize the recognizer
config = mmengine.Config.fromfile(config)
# build the model from a config file and a checkpoint file
model = init_recognizer(config, checkpoint, device)

# Get Action classification results.
result = inference_skeleton(model, pose_results, (h, w))

Loads checkpoint by http backend from path: https://download.openmmlab.com/mmaction/skeleton/posec3d/slowonly_r50_u48_240e_ntu60_xsub_keypoint/slowonly_r50_u48_240e_ntu60_xsub_keypoint-f3adabf1.pth


In [14]:
# find the index of highest predicted score on result
max_pred_index = result.pred_score.argmax().item()

label_map = [x.strip() for x in open(label_map).readlines()]

# set the highest predicted label as action_label
action_label = label_map[max_pred_index]
print(action_label)

staggering


In [15]:
visualize(pose_config, out_filename, frames, pose_data_samples, action_label)

tmp_dir.cleanup()

Drawing skeleton for each frame
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 319/319, 273.2 task/s, elapsed: 1s, ETA:     0s
Moviepy - Building video data/uji_jalan_out_demo.mp4.
Moviepy - Writing video data/uji_jalan_out_demo.mp4



Moviepy - Done !
Moviepy - video ready data/uji_jalan_out_demo.mp4


In [16]:
HTML('<video width=50% controls autoplay loop><source src="data/uji_jalan.mp4"></video>')

In [17]:
HTML('<video width=50% controls autoplay loop><source src="data/uji_jalan_out_demo.mp4"></video>')

## Testing recognition from bash - 2019

In [2]:
!python mmaction2/demo/demo_skeleton.py data/uji_jalan.mp4 data/uji_jalan_out_2019.mp4 \
    --config mmaction2/configs/skeleton/posec3d/slowonly_r50_u48_240e_ntu120_xsub_keypoint.py \
    --checkpoint work_dirs/slowonly_r50_u48_240e_ntu120_xsub_keypoint/best_top1_acc_epoch_90_8.pth \
    --det-config mmaction2/demo/demo_configs/faster-rcnn_r50_fpn_2x_coco_infer.py \
    --det-checkpoint http://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_2x_coco/faster_rcnn_r50_fpn_2x_coco_bbox_mAP-0.384_20200504_210434-a5d8aa15.pth \
    --det-score-thr 0.9 \
    --det-cat-id 0 \
    --pose-config mmaction2/demo/demo_configs/td-hm_hrnet-w32_8xb64-210e_coco-256x192_infer.py \
    --pose-checkpoint https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w32_coco_256x192-c78dce93_20200708.pth \
    --label-map mmaction2/tools/data/skeleton/label_5.txt

Loads checkpoint by http backend from path: http://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_2x_coco/faster_rcnn_r50_fpn_2x_coco_bbox_mAP-0.384_20200504_210434-a5d8aa15.pth
Performing Human Detection for each frame
[                                                  ] 0/319, elapsed: 0s, ETA:02/19 00:48:45 - mmengine - WARNING - "FileClient" will be deprecated in future. Please use io functions in https://mmengine.readthedocs.io/en/latest/api/fileio.html#file-io
02/19 00:48:45 - mmengine - WARNING - "HardDiskBackend" is the alias of "LocalBackend" and the former will be deprecated in future.
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 319/319, 5.0 task/s, elapsed: 64s, ETA:     0s
Loads checkpoint by http backend from path: https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w32_coco_256x192-c78dce93_20200708.pth
Performing Human Pose Estimation for each frame
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 319/319, 14.7 task/s, elapsed: 22s, ETA:     0s
Loads checkpoint by lo

In [19]:
HTML('<video width=50% controls autoplay loop><source src="data/uji_jalan.mp4"></video>')

In [ ]:
HTML('<video width=50% controls autoplay loop><source src="data/uji_jalan_out_2019.mp4"></video>')

## Testing recognition from python - 2019

In [3]:
FONTFACE = cv2.FONT_HERSHEY_DUPLEX
FONTSCALE = 0.75
FONTCOLOR = (255, 255, 255)  # BGR, white
THICKNESS = 1
LINETYPE = 1

In [4]:
def visualize(pose_config, out_filename, frames, data_samples, action_label):
    pose_config = mmengine.Config.fromfile(pose_config)
    visualizer = VISUALIZERS.build(pose_config.visualizer)
    visualizer.set_dataset_meta(data_samples[0].dataset_meta)

    vis_frames = []
    print('Drawing skeleton for each frame')
    for d, f in track_iter_progress(list(zip(data_samples, frames))):
        f = mmcv.imconvert(f, 'bgr', 'rgb')
        visualizer.add_datasample(
            'result',
            f,
            data_sample=d,
            draw_gt=False,
            draw_heatmap=False,
            draw_bbox=True,
            show=False,
            wait_time=0,
            out_file=None,
            kpt_thr=0.3)
        vis_frame = visualizer.get_image()
        cv2.putText(vis_frame, action_label, (10, 30), FONTFACE, FONTSCALE,
                    FONTCOLOR, THICKNESS, LINETYPE)
        vis_frames.append(vis_frame)

    vid = mpy.ImageSequenceClip(vis_frames, fps=24)
    vid.write_videofile(out_filename, remove_temp=True)

In [5]:
video = 'data/uji_jalan.mp4'
out_filename = 'data/uji_jalan_out_2019.mp4'

# Choose to use an action classification config
config = 'mmaction2/configs/skeleton/posec3d/slowonly_r50_u48_240e_ntu120_xsub_keypoint.py'
# Setup a checkpoint file to load
checkpoint = 'work_dirs/slowonly_r50_u48_240e_ntu120_xsub_keypoint/best_top1_acc_epoch_90_8.pth' #class 5 label

# human detection config
det_config = 'mmaction2/demo/demo_configs/faster-rcnn_r50_fpn_2x_coco_infer.py'
det_checkpoint = 'http://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_2x_coco/faster_rcnn_r50_fpn_2x_coco_bbox_mAP-0.384_20200504_210434-a5d8aa15.pth'
det_score_thr = 0.9
det_cat_id = 0

# pose estimation config
pose_config = 'mmaction2/demo/demo_configs/td-hm_hrnet-w32_8xb64-210e_coco-256x192_infer.py'
pose_checkpoint = 'https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w32_coco_256x192-c78dce93_20200708.pth'
label_map = 'mmaction2/tools/data/skeleton/label_5.txt'

In [6]:
device = 'cuda:0'
short_side = 480

In [7]:
tmp_dir = tempfile.TemporaryDirectory()
frame_paths, frames = frame_extract(video, short_side,
                                    tmp_dir.name)

h, w, _ = frames[0].shape

In [8]:
# Get Human detection results.
det_results, _ = detection_inference(det_config, det_checkpoint,
                                     frame_paths, det_score_thr,
                                     det_cat_id, device)
torch.cuda.empty_cache()

Loads checkpoint by http backend from path: http://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_2x_coco/faster_rcnn_r50_fpn_2x_coco_bbox_mAP-0.384_20200504_210434-a5d8aa15.pth
Performing Human Detection for each frame
[                                                  ] 0/319, elapsed: 0s, ETA:02/19 01:33:50 - mmengine - WARNING - "FileClient" will be deprecated in future. Please use io functions in https://mmengine.readthedocs.io/en/latest/api/fileio.html#file-io
02/19 01:33:50 - mmengine - WARNING - "HardDiskBackend" is the alias of "LocalBackend" and the former will be deprecated in future.
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 319/319, 5.0 task/s, elapsed: 64s, ETA:     0s


In [9]:
# Get Pose estimation results.
pose_results, pose_data_samples = pose_inference(pose_config,
                                                 pose_checkpoint,
                                                 frame_paths, det_results,
                                                 device)
torch.cuda.empty_cache()

Loads checkpoint by http backend from path: https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w32_coco_256x192-c78dce93_20200708.pth
Performing Human Pose Estimation for each frame
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 319/319, 15.4 task/s, elapsed: 21s, ETA:     0s


In [13]:
# Initialize the recognizer
config = mmengine.Config.fromfile(config)
# build the model from a config file and a checkpoint file
model = init_recognizer(config, checkpoint, device)

# Get Action classification results.
result = inference_skeleton(model, pose_results, (h, w))

Loads checkpoint by local backend from path: work_dirs/slowonly_r50_u48_240e_ntu120_xsub_keypoint/best_top1_acc_epoch_90_8.pth


KeyError: 'Collect is not in the mmaction::transform registry. Please check whether the value of `Collect` is correct or it was registered as expected. More details can be found at https://mmengine.readthedocs.io/en/latest/advanced_tutorials/config.html#import-the-custom-module'

In [26]:
config = 'mmaction2/configs/skeleton/posec3d/slowonly_r50_u48_240e_ntu120_xsub_keypoint.py'
checkpoint = "work_dirs/slowonly_r50_u48_240e_ntu120_xsub_keypoint/best_top1_acc_epoch_90_8.pth"

config = mmengine.Config.fromfile(config)

# build the model from a config file and a checkpoint file
model = init_recognizer(config, checkpoint, device)

# Get Action classification results.
result = inference_skeleton(model, pose_results, (h, w))

Loads checkpoint by local backend from path: work_dirs/slowonly_r50_u48_240e_ntu120_xsub_keypoint/best_top1_acc_epoch_90_8.pth


OutOfMemoryError: CUDA out of memory. Tried to allocate 256.00 MiB. GPU 0 has a total capacty of 3.81 GiB of which 132.31 MiB is free. Including non-PyTorch memory, this process has 3.67 GiB memory in use. Of the allocated memory 2.82 GiB is allocated by PyTorch, and 785.29 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [23]:
# find the index of highest predicted score on result
max_pred_index = result.pred_score.argmax().item()

label_map = [x.strip() for x in open(label_map).readlines()]

# set the highest predicted label as action_label
action_label = label_map[max_pred_index]
print(action_label)

IndexError: list index out of range

In [ ]:
visualize(pose_config, out_filename, frames, pose_data_samples, action_label)

tmp_dir.cleanup()

In [ ]:
HTML('<video width=50% controls autoplay loop><source src="data/manipulasi6.mp4"></video>')

## Testing recognition + detection from bash - DEMO

In [ ]:
!python mmaction2/demo/demo_video_structuralize.py \
    --video mmaction2/demo/test_video_structuralize.mp4 \
    --out-filename mmaction2/demo/test_stdet_recognition_output.mp4 \
\
    --rgb-stdet-config mmaction2/configs/detection/slowonly/slowonly_kinetics400-pretrained-r101_8xb16-8x8x1-20e_ava21-rgb.py \
    --skeleton-stdet-checkpoint https://download.openmmlab.com/mmaction/skeleton/posec3d/posec3d_ava.pth \
\
    --det-config mmaction2/demo/demo_configs/faster-rcnn_r50_fpn_2x_coco_infer.py \
    --det-checkpoint http://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_2x_coco/faster_rcnn_r50_fpn_2x_coco_bbox_mAP-0.384_20200504_210434-a5d8aa15.pth \
\
    --pose-config mmaction2/demo/demo_configs/td-hm_hrnet-w32_8xb64-210e_coco-256x192_infer.py \
    --pose-checkpoint https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w32_coco_256x192-c78dce93_20200708.pth \
\
    --skeleton-config mmaction2/configs/skeleton/posec3d/slowonly_r50_8xb16-u48-240e_ntu60-xsub-keypoint.py \
    --skeleton-checkpoint https://download.openmmlab.com/mmaction/skeleton/posec3d/posec3d_k400.pth \
\
    --use-skeleton-stdet \
    --use-skeleton-recog \
\
    --label-map-stdet mmaction2/tools/data/ava/label_map.txt \
    --label-map mmaction2/tools/data/kinetics/label_map_k400.txt

In [ ]:
HTML('<video width=50% controls autoplay loop><source src="mmaction2/demo/test_video_structuralize.mp4"></video>')

In [ ]:
HTML('<video width=50% controls autoplay loop><source src="mmaction2/demo/test_stdet_recognition_output.mp4"></video>')

In [ ]:
!python mmaction2/demo/demo_video_structuralize.py \
    --video mmaction2/demo/demo_skeleton.mp4 \
    --out-filename mmaction2/demo/demo_skeleton_vs_out.mp4 \
    --rgb-stdet-config mmaction2/configs/detection/slowonly/slowonly_kinetics400-pretrained-r101_8xb16-8x8x1-20e_ava21-rgb.py \
    --skeleton-stdet-checkpoint https://download.openmmlab.com/mmaction/skeleton/posec3d/posec3d_ava.pth \
    --det-config mmaction2/demo/demo_configs/faster-rcnn_r50_fpn_2x_coco_infer.py \
    --det-checkpoint http://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_2x_coco/faster_rcnn_r50_fpn_2x_coco_bbox_mAP-0.384_20200504_210434-a5d8aa15.pth \
    --pose-config mmaction2/demo/demo_configs/td-hm_hrnet-w32_8xb64-210e_coco-256x192_infer.py \
    --pose-checkpoint https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w32_coco_256x192-c78dce93_20200708.pth \
    --skeleton-config mmaction2/configs/skeleton/posec3d/slowonly_r50_8xb16-u48-240e_ntu60-xsub-keypoint.py \
    --skeleton-checkpoint https://download.openmmlab.com/mmaction/skeleton/posec3d/posec3d_k400.pth \
    --use-skeleton-stdet \
    --use-skeleton-recog \
    --label-map-stdet mmaction2/tools/data/ava/label_map.txt \
    --label-map mmaction2/tools/data/kinetics/label_map_k400.txt

In [ ]:
HTML('<video width=50% controls autoplay loop><source src="mmaction2/demo/demo_skeleton.mp4"></video>')

In [ ]:
HTML('<video width=50% controls autoplay loop><source src="mmaction2/demo/demo_skeleton_vs_out.mp4"></video>')

In [1]:
!python mmaction2/demo/demo_video_structuralize.py \
    --video data/DJI_0011.MP4 \
    --out-filename data/DJI_0011_out.MP4 \
\
    --rgb-stdet-config mmaction2/configs/detection/slowonly/slowonly_kinetics400-pretrained-r101_8xb16-8x8x1-20e_ava21-rgb.py \
    --skeleton-stdet-checkpoint https://download.openmmlab.com/mmaction/skeleton/posec3d/posec3d_ava.pth \
\
    --det-config mmaction2/demo/demo_configs/faster-rcnn_r50_fpn_2x_coco_infer.py \
    --det-checkpoint http://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_2x_coco/faster_rcnn_r50_fpn_2x_coco_bbox_mAP-0.384_20200504_210434-a5d8aa15.pth \
\
    --pose-config mmaction2/demo/demo_configs/td-hm_hrnet-w32_8xb64-210e_coco-256x192_infer.py \
    --pose-checkpoint https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w32_coco_256x192-c78dce93_20200708.pth \
\
    --skeleton-config mmaction2/configs/skeleton/posec3d/slowonly_r50_8xb16-u48-240e_ntu60-xsub-keypoint.py \
    --skeleton-checkpoint https://download.openmmlab.com/mmaction/skeleton/posec3d/posec3d_k400.pth \
\
    --use-skeleton-stdet \
    --use-skeleton-recog \
\
    --label-map-stdet mmaction2/tools/data/ava/label_map.txt \
    --label-map mmaction2/tools/data/kinetics/label_map_k400.txt

In [ ]:
HTML('<video width=50% controls autoplay loop><source src="mmaction2/demo/demo_skeleton.mp4"></video>')

In [ ]:
HTML('<video width=50% controls autoplay loop><source src="mmaction2/demo/demo_skeleton_vs_out.mp4"></video>')

## Testing recognition + detection from python - DEMO